In [16]:
import tensorflow as tf
import numpy as np
import os
import scipy.io as spio
from matplotlib import pyplot as plt
from imageio import imread

#### Obs: If you only have one of the datasets (does not matter which one), just run all the notebook's cells and it will work just fine.

In [17]:
# define base paths for pascal the original VOC dataset training images
base_dataset_dir_voc = '/Users/udaysawhney/Desktop/segmentation/deeplab_v3-master/dataset/VOCdevkit/VOC2012/'
images_folder_name_voc = "JPEGImages/"
annotations_folder_name_voc = "SegmentationClass/"
images_dir_voc = os.path.join(base_dataset_dir_voc, images_folder_name_voc)
print(images_dir_voc)
annotations_dir_voc = os.path.join(base_dataset_dir_voc, annotations_folder_name_voc)

/Users/udaysawhney/Desktop/segmentation/deeplab_v3-master/dataset/VOCdevkit/VOC2012/JPEGImages/


In [18]:
# define base paths for pascal augmented VOC images
# download: http://home.bharathh.info/pubs/codes/SBD/download.html
base_dataset_dir_aug_voc = '<pascal/augmented/VOC/images/path>/benchmark_RELEASE/dataset'
images_folder_name_aug_voc = "img/"
annotations_folder_name_aug_voc = "cls/"
images_dir_aug_voc = os.path.join(base_dataset_dir_aug_voc, images_folder_name_aug_voc)
annotations_dir_aug_voc = os.path.join(base_dataset_dir_aug_voc, annotations_folder_name_aug_voc)

In [19]:
def get_files_list(base_dataset_dir, images_folder_name, annotations_folder_name, filename):
    images_dir = os.path.join(base_dataset_dir, images_folder_name)
    annotations_dir = os.path.join(base_dataset_dir, annotations_folder_name)

    file = open(filename, 'r')
    images_filename_list = [line for line in file]
    return images_filename_list

In [21]:
images_filename_list = get_files_list(base_dataset_dir_voc, images_folder_name_voc, annotations_folder_name_voc, "custom_train.txt")
print("Total number of training images:", len(images_filename_list))

Total number of training images: 8252


In [8]:
# shuffle array and separate 10% to validation
np.random.shuffle(images_filename_list)
val_images_filename_list = images_filename_list[:int(0.10*len(images_filename_list))]
train_images_filename_list = images_filename_list[int(0.10*len(images_filename_list)):]

In [9]:
print("train set size:", len(train_images_filename_list))
print("val set size:", len(val_images_filename_list))

train set size: 7427
val set size: 825


In [10]:
TRAIN_DATASET_DIR="./tfrecords/"
if not os.path.exists(TRAIN_DATASET_DIR):
    os.mkdir(TRAIN_DATASET_DIR)
    
TRAIN_FILE = 'train.tfrecords'
VALIDATION_FILE = 'validation.tfrecords'
train_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,TRAIN_FILE))
val_writer = tf.python_io.TFRecordWriter(os.path.join(TRAIN_DATASET_DIR,VALIDATION_FILE))

In [11]:
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [12]:
def read_annotation_from_mat_file(annotations_dir, image_name):
    annotations_path = os.path.join(annotations_dir, (image_name.strip() + ".xml"))
    mat = spio.loadmat(annotations_path)
    img = mat['GTcls']['Segmentation'][0][0]
    return img

In [13]:
for i, image_name in enumerate(train_images_filename_list):
    image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))

In [14]:
def create_tfrecord_dataset(filename_list, writer):

    # create training tfrecord
    read_imgs_counter = 0
    for i, image_name in enumerate(filename_list):
        image_name = str(image_name)

        try:
            image_np = imread(os.path.join(images_dir_aug_voc, image_name.strip() + ".jpg"))
        except FileNotFoundError:
            try:
                # read from Pascal VOC path
                image_np = imread(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
            except FileNotFoundError:
                print(os.path.join(images_dir_voc, image_name.strip() + ".jpg"))
                print("File:",image_name.strip(),"not found.")
                continue
        try:
            annotation_np = read_annotation_from_mat_file(annotations_dir_aug_voc, image_name)
        except FileNotFoundError:
            # read from Pascal VOC path
            try:
                annotation_np = imread(os.path.join(annotations_dir_voc, image_name.strip() + ".png"))
                print('SUCCESS!!')
            except FileNotFoundError:
                print("File:",image_name.strip(),"not found lolololol.")
                continue
            
        read_imgs_counter += 1
        image_h = image_np.shape[0]
        image_w = image_np.shape[1]

        img_raw = image_np.tostring()
        annotation_raw = annotation_np.tostring()

        example = tf.train.Example(features=tf.train.Features(feature={
                'height': _int64_feature(image_h),
                'width': _int64_feature(image_w),
                'image_raw': _bytes_feature(img_raw),
                'annotation_raw': _bytes_feature(annotation_raw)}))

        writer.write(example.SerializeToString())
    
    print("End of TfRecord. Total of image written:", read_imgs_counter)
    writer.close()

In [15]:
# create training dataset
create_tfrecord_dataset(train_images_filename_list, train_writer)

File: 2008_003593 not found lolololol.
File: 2010_000617 not found lolololol.
File: 2009_002301 not found lolololol.
SUCCESS!!
File: 2008_003160 not found lolololol.
SUCCESS!!
File: 2010_004816 not found lolololol.
File: 2011_002943 not found lolololol.
File: 2008_006831 not found lolololol.
File: 2008_003437 not found lolololol.
File: 2008_003763 not found lolololol.
File: 2008_004074 not found lolololol.
File: 2009_004175 not found lolololol.
SUCCESS!!
File: 2010_003987 not found lolololol.
File: 2008_000196 not found lolololol.
File: 2008_001047 not found lolololol.
File: 2010_004257 not found lolololol.
File: 2008_003469 not found lolololol.
File: 2009_002714 not found lolololol.
File: 2010_002223 not found lolololol.
File: 2008_007486 not found lolololol.
File: 2008_001134 not found lolololol.
File: 2008_001694 not found lolololol.
File: 2008_008341 not found lolololol.
File: 2008_005071 not found lolololol.
File: 2008_004087 not found lolololol.
File: 2008_000488 not found lololo

File: 2008_005043 not found lolololol.
File: 2008_004371 not found lolololol.
File: 2010_005608 not found lolololol.
File: 2008_007281 not found lolololol.
File: 2011_002388 not found lolololol.
File: 2011_003149 not found lolololol.
File: 2010_004849 not found lolololol.
SUCCESS!!
File: 2009_003441 not found lolololol.
File: 2008_000689 not found lolololol.
File: 2010_004712 not found lolololol.
File: 2010_003779 not found lolololol.
File: 2008_004745 not found lolololol.
SUCCESS!!
File: 2008_008132 not found lolololol.
SUCCESS!!
File: 2009_004593 not found lolololol.
File: 2008_003697 not found lolololol.
File: 2010_001846 not found lolololol.
File: 2010_002642 not found lolololol.
File: 2010_002487 not found lolololol.
SUCCESS!!
File: 2008_001582 not found lolololol.
File: 2009_001349 not found lolololol.
File: 2010_005959 not found lolololol.
File: 2009_001917 not found lolololol.
File: 2008_000531 not found lolololol.
File: 2008_002031 not found lolololol.
SUCCESS!!
File: 2009_001

File: 2008_004497 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2010_003186 not found lolololol.
File: 2009_002214 not found lolololol.
File: 2008_008002 not found lolololol.
File: 2009_004012 not found lolololol.
File: 2009_004642 not found lolololol.
File: 2008_008048 not found lolololol.
File: 2010_003906 not found lolololol.
File: 2009_002688 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2010_003645 not found lolololol.
File: 2009_001241 not found lolololol.
File: 2010_003025 not found lolololol.
File: 2010_003461 not found lolololol.
File: 2010_002834 not found lolololol.
File: 2011_001168 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2011_002022 not found lolololol.
File: 2009_000247 not found lolololol.
SUCCESS!!
File: 2008_003533 not found lolololol.
File: 2008_000342 not found lolololol.
File: 2009_002093 not found lolololol.
File: 2010_003259 not found lolololol.
SUCCESS!!
File: 2010_004691 not found lolololol.
File: 2008_004770 not found lolololol.
File: 2009_000327 not f

File: 2008_007835 not found lolololol.
File: 2010_003709 not found lolololol.
File: 2009_003790 not found lolololol.
SUCCESS!!
File: 2008_008453 not found lolololol.
SUCCESS!!
File: 2010_004048 not found lolololol.
File: 2010_004944 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2008_004208 not found lolololol.
SUCCESS!!
File: 2010_001863 not found lolololol.
File: 2008_006403 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2010_006078 not found lolololol.
File: 2010_004477 not found lolololol.
File: 2010_003719 not found lolololol.
File: 2011_003023 not found lolololol.
File: 2011_002897 not found lolololol.
File: 2011_000748 not found lolololol.
File: 2010_000439 not found lolololol.
SUCCESS!!
File: 2010_001583 not found lolololol.
SUCCESS!!
File: 2011_003044 not found lolololol.
File: 2009_004525 not found lolololol.
File: 2010_004009 not found lolololol.
SUCCESS!!
File: 2008_001299 not found lolololol.
SUCCESS!!
File: 2009_005094 not found lolololol.
SUCCESS!!
File: 2011_001111 not 

SUCCESS!!
File: 2011_001216 not found lolololol.
File: 2011_002933 not found lolololol.
SUCCESS!!
File: 2010_005100 not found lolololol.
File: 2011_000077 not found lolololol.
File: 2010_002734 not found lolololol.
File: 2008_007274 not found lolololol.
File: 2009_004200 not found lolololol.
File: 2008_001871 not found lolololol.
File: 2010_001450 not found lolololol.
File: 2008_006400 not found lolololol.
File: 2010_004576 not found lolololol.
SUCCESS!!
File: 2009_002784 not found lolololol.
File: 2008_005423 not found lolololol.
File: 2009_001245 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2009_002853 not found lolololol.
File: 2010_000103 not found lolololol.
File: 2009_001237 not found lolololol.
File: 2010_004660 not found lolololol.
File: 2008_000756 not found lolololol.
File: 2008_008528 not found lolololol.
File: 2011_001757 not found lolololol.
File: 2009_004841 not found lolololol.
File: 2009_001931 not found lolololol.
File: 2008_003671 not found lolololol.
File: 2008_002

...

File: 2008_007583 not found lolololol.
File: 2008_002148 not found lolololol.
File: 2008_003609 not found lolololol.
File: 2010_003527 not found lolololol.
File: 2008_001834 not found lolololol.
SUCCESS!!
File: 2010_001311 not found lolololol.
File: 2009_000930 not found lolololol.
File: 2010_002621 not found lolololol.
SUCCESS!!
File: 2010_001066 not found lolololol.
File: 2008_004955 not found lolololol.
File: 2008_007409 not found lolololol.
File: 2009_000344 not found lolololol.
File: 2010_001771 not found lolololol.
File: 2009_003639 not found lolololol.
SUCCESS!!
SUCCESS!!
SUCCESS!!
File: 2009_001779 not found lolololol.
File: 2008_004545 not found lolololol.
File: 2008_008097 not found lolololol.
File: 2008_000742 not found lolololol.
File: 2008_001267 not found lolololol.
File: 2010_004918 not found lolololol.
File: 2010_000872 not found lolololol.
File: 2009_002937 not found lolololol.
File: 2008_003870 not found lolololol.
File: 2009_001199 not found lolololol.
File: 2008_002

SUCCESS!!
File: 2008_003303 not found lolololol.
File: 2009_001897 not found lolololol.
File: 2010_002692 not found lolololol.
File: 2008_004398 not found lolololol.
File: 2008_006767 not found lolololol.
SUCCESS!!
File: 2009_001910 not found lolololol.
File: 2008_002329 not found lolololol.
File: 2010_000658 not found lolololol.
File: 2008_005639 not found lolololol.
File: 2008_002368 not found lolololol.
File: 2008_005253 not found lolololol.
File: 2008_005794 not found lolololol.
File: 2009_004322 not found lolololol.
File: 2009_000145 not found lolololol.
File: 2010_004021 not found lolololol.
File: 2009_000971 not found lolololol.
File: 2010_001731 not found lolololol.
File: 2011_002921 not found lolololol.
File: 2009_002429 not found lolololol.
File: 2008_003143 not found lolololol.
File: 2008_005429 not found lolololol.
SUCCESS!!
File: 2010_005193 not found lolololol.
File: 2008_005066 not found lolololol.
File: 2009_001800 not found lolololol.
File: 2011_001647 not found lololo

File: 2010_001325 not found lolololol.
File: 2010_005349 not found lolololol.
File: 2008_008635 not found lolololol.
File: 2008_008231 not found lolololol.
SUCCESS!!
File: 2008_007473 not found lolololol.
File: 2008_006102 not found lolololol.
File: 2009_002877 not found lolololol.
File: 2010_001795 not found lolololol.
File: 2009_003259 not found lolololol.
File: 2008_002401 not found lolololol.
File: 2009_000529 not found lolololol.
File: 2009_003958 not found lolololol.
File: 2009_003183 not found lolololol.
File: 2011_002638 not found lolololol.
File: 2008_003974 not found lolololol.
SUCCESS!!
File: 2010_001739 not found lolololol.
File: 2009_000297 not found lolololol.
File: 2009_003913 not found lolololol.
File: 2011_002661 not found lolololol.
File: 2011_002925 not found lolololol.
File: 2008_008547 not found lolololol.
File: 2010_000926 not found lolololol.
File: 2009_002676 not found lolololol.
File: 2008_007167 not found lolololol.
File: 2008_000132 not found lolololol.
File:

File: 2010_001054 not found lolololol.
File: 2008_002698 not found lolololol.
File: 2008_008725 not found lolololol.
File: 2008_003635 not found lolololol.
File: 2008_007761 not found lolololol.
SUCCESS!!
File: 2008_007085 not found lolololol.
File: 2009_004429 not found lolololol.
File: 2010_000473 not found lolololol.
File: 2011_000851 not found lolololol.
File: 2010_003159 not found lolololol.
File: 2011_001847 not found lolololol.
File: 2010_005557 not found lolololol.
File: 2010_006015 not found lolololol.
File: 2009_002977 not found lolololol.
File: 2009_003847 not found lolololol.
File: 2008_002549 not found lolololol.
SUCCESS!!
File: 2008_006429 not found lolololol.
File: 2008_007837 not found lolololol.
File: 2009_005165 not found lolololol.
File: 2010_003734 not found lolololol.
File: 2008_002013 not found lolololol.
File: 2008_003753 not found lolololol.
File: 2008_001161 not found lolololol.
File: 2008_000421 not found lolololol.
SUCCESS!!
File: 2010_003303 not found lololo

SUCCESS!!
File: 2011_001066 not found lolololol.
File: 2010_001649 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2008_002801 not found lolololol.
File: 2010_000743 not found lolololol.
SUCCESS!!
File: 2009_002044 not found lolololol.
File: 2008_001448 not found lolololol.
File: 2010_001465 not found lolololol.
File: 2009_001254 not found lolololol.
File: 2010_002378 not found lolololol.
SUCCESS!!
File: 2008_008055 not found lolololol.
File: 2011_000859 not found lolololol.
File: 2011_000520 not found lolololol.
File: 2008_005881 not found lolololol.
SUCCESS!!
SUCCESS!!
File: 2010_001669 not found lolololol.
File: 2009_000796 not found lolololol.
File: 2008_000095 not found lolololol.
File: 2008_004964 not found lolololol.
SUCCESS!!
File: 2010_001885 not found lolololol.
SUCCESS!!
File: 2009_004141 not found lolololol.
File: 2008_000761 not found lolololol.
File: 2010_004609 not found lolololol.
File: 2008_008735 not found lolololol.
File: 2010_001211 not found lolololol.
File: 2009_00

In [27]:
# create validation dataset
create_tfrecord_dataset(val_images_filename_list, val_writer)

File: 2009_000303 not found.
File: 2008_005599 not found.
File: 2009_002685 not found.
File: 2008_006753 not found.
File: 2010_002856 not found.
File: 2011_002988 not found.
File: 2007_005227 not found.
File: 2009_004953 not found.
File: 2010_004874 not found.
File: 2008_001876 not found.
File: 2008_005379 not found.
File: 2008_006747 not found.
File: 2009_004374 not found.
File: 2008_007525 not found.
File: 2010_001044 not found.
File: 2010_003139 not found.
File: 2011_002571 not found.
File: 2007_001698 not found.
File: 2010_005937 not found.
File: 2010_000689 not found.
File: 2010_002552 not found.
File: 2009_001500 not found.
File: 2008_005354 not found.
File: 2009_001413 not found.
File: 2007_003000 not found.
File: 2009_003636 not found.
File: 2010_001625 not found.
File: 2008_002177 not found.
File: 2010_000485 not found.
File: 2008_005541 not found.
File: 2010_001277 not found.
File: 2008_003675 not found.
File: 2010_003672 not found.
File: 2011_001949 not found.
File: 2008_008

File: 2009_003039 not found.
File: 2008_008057 not found.
File: 2009_000157 not found.
File: 2010_004597 not found.
File: 2008_000142 not found.
File: 2009_000337 not found.
File: 2011_002236 not found.
File: 2008_007947 not found.
File: 2011_002598 not found.
File: 2008_007054 not found.
File: 2009_002465 not found.
File: 2010_000613 not found.
File: 2008_005685 not found.
File: 2008_005634 not found.
File: 2008_007287 not found.
File: 2009_001085 not found.
File: 2008_001257 not found.
File: 2008_008180 not found.
File: 2008_003534 not found.
File: 2008_006625 not found.
File: 2010_003801 not found.
File: 2011_001399 not found.
File: 2008_007599 not found.
File: 2008_004427 not found.
File: 2010_001119 not found.
File: 2009_004587 not found.
File: 2008_002562 not found.
File: 2011_003151 not found.
File: 2008_005309 not found.
File: 2008_006969 not found.
File: 2008_006808 not found.
File: 2010_004445 not found.
File: 2011_000765 not found.
File: 2010_002456 not found.
File: 2008_006

File: 2008_001691 not found.
File: 2008_003745 not found.
File: 2009_002519 not found.
File: 2010_003887 not found.
File: 2010_003651 not found.
File: 2009_003168 not found.
File: 2009_003668 not found.
File: 2011_001885 not found.
File: 2011_002291 not found.
File: 2008_003151 not found.
File: 2008_006211 not found.
File: 2008_005362 not found.
File: 2010_002457 not found.
File: 2011_002039 not found.
File: 2009_001395 not found.
File: 2010_000386 not found.
File: 2010_002185 not found.
File: 2007_004500 not found.
File: 2009_004096 not found.
File: 2008_006750 not found.
File: 2008_002930 not found.
File: 2009_003351 not found.
File: 2009_001635 not found.
File: 2008_005677 not found.
File: 2008_002961 not found.
File: 2010_001689 not found.
File: 2009_000486 not found.
File: 2008_003417 not found.
File: 2008_000723 not found.
File: 2008_004110 not found.
File: 2008_003449 not found.
File: 2010_004717 not found.
File: 2009_002879 not found.
File: 2008_005136 not found.
File: 2009_000

In [ ]:
LOG_FOLDER = './tboard_logs'
TEST_DATASET_DIR="./dataset/tfrecords"
TEST_FILE = 'test123.tfrecords'

test_filenames = [os.path.join(TEST_DATASET_DIR,TEST_FILE)]
test_dataset = tf.data.TFRecordDataset(test_filenames)
test_dataset = test_dataset.map(tf_record_parser)  # Parse the record into tensors.
print(test_dataset)